# Домашнее задание 1
## Harry Potter and the Action Prediction Challenge from Natural Language

0. Подготовка

Прежде всего необходимо установить и загрузить некоторые необходимые для выполнения задания модули 

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import random
import nltk
from nltk.corpus import stopwords
import string
from tqdm import tqdm
import pickle
from collections import Counter
from nltk.tag import StanfordNERTagger
import itertools

In [2]:
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/care1e55/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Скачать датасеты
# wget https://www.dropbox.com/s/23xet9kvbqna1qs/hpac_raw.zip?dl=0
# wget https://www.dropbox.com/s/gwfgmomdbetvdye/hpac_lower_tokenized.zip?dl=0
# wget https://www.dropbox.com/s/3vdz0mouvex8abd/hpac_splits.zip?dl=0

In [4]:
base_path = 'corpus'
raw_corpus_path = os.path.join(base_path, 'fanfiction_texts')
tokenized_corpus_path = os.path.join(base_path, 'hpac_source')
split_corpus_path = os.path.join(base_path, 'hpac_corpus')

Посмотрим на первую строку случайных файлов корпуса

In [5]:
raw_text_file = random.choice(os.listdir(raw_corpus_path))
with open(os.path.join(raw_corpus_path, raw_text_file)) as f:
    for line in f.readlines():
        break

In [6]:
raw_text_file = random.choice(os.listdir(tokenized_corpus_path))
with open(os.path.join(raw_corpus_path, raw_text_file)) as f:
    for line in f.readlines():
        print(line)
        break

Disclaimer:  I don't own Harry Potter or the wonderful magical world in which he lives. I'm just having fun. Note:  THANKS to my betas on this story: kazfeist and eilonwy! This story was a lot of fun, I hope you like it! ooo 



## Часть 1. Эксплоративный анализ
1. Найдите топ-1000 слов по частоте без учета стоп-слов.

Подсчет частот всех попдающихся слов с помощью акумулирующего Counter. Потом top 1000 можно будет получить вызвав метод most_common(1000)

In [30]:
top_1000 = Counter()
for raw_text_file in tqdm(os.listdir(tokenized_corpus_path)):
    with open(os.path.join(tokenized_corpus_path, raw_text_file)) as f:
        for line in f:
            top_1000 += Counter(line.split())
#         break

100%|██████████| 36225/36225 [19:22<00:00, 31.16it/s]


In [35]:
with open("top_1000.pckl", 'wb') as file:
    pickle.dump(top_1000, file)

In [3]:
with open("top_1000.pckl", 'rb') as file:
    top_1000 = pickle.load(file)

In [9]:
# filter stopwords and word with punctuation
top_1000_filtered = { 
    word: count for word, count in tqdm(top_1000.items()) 
    if word not in stopwords.words("english")
    and not any(punct in word for punct in string.punctuation)
}

100%|██████████| 613880/613880 [00:44<00:00, 13769.52it/s]


Вывод 1000 слов займет слишком много места. Пока что выведем 10, а позже 1000 понадобится для визуализации эмбедингов

In [11]:
# Counter(top_1000_filtered).most_common(1000)
Counter(top_1000_filtered).most_common(10)

[('harry', 3991017),
 ('said', 2262072),
 ('would', 1903185),
 ('hermione', 1826879),
 ('could', 1687864),
 ('back', 1396452),
 ('draco', 1386180),
 ('one', 1376314),
 ('like', 1256561),
 ('know', 1179021)]

2. Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия.

Понадобятся функции для фильтрации и итератор возвращающий текущее слово и предыдущее. Вторая функциия нужна что бы искать пары слов

In [6]:
def biwords(words):
    for i, word in enumerate(words, 1):
        try:
            yield word, words[i]
        except IndexError:
            pass

In [7]:
def filter_stopwords_and_punkt(tokens):
    return list(filter(
        lambda word: 
            word.lower() not in stopwords.words("english") 
            and not any(punct in word for punct in string.punctuation)
        , tokens
    ))

Найдем все слова начинающиеся с большой буквы, пары таких слов и со словом professor перед ним

In [9]:
names = []
full_names = []
professor_names = []

for raw_text_file in tqdm(os.listdir(raw_corpus_path)):
    with open(os.path.join(raw_corpus_path, raw_text_file)) as f:
        for line in f:
            tokens = nltk.word_tokenize(line)
            for token_pair in biwords(tokens):
                if token_pair[0].istitle():
                    names.append(token_pair[0])
                    if token_pair[1].istitle(): full_names.append(' '.join(token_pair))
                    if token_pair[0].lower()=='professor' and token_pair[1].istitle(): 
                        professor_names.append(' '.join(token_pair))

In [92]:
# Сохраним
# with open("names.pckl", 'wb') as file:
#     pickle.dump(names, file)
# with open("full_names.pckl", 'wb') as file:
#     pickle.dump(full_names, file)
# with open("professor_names.pckl", 'wb') as file:
#     pickle.dump(professor_names, file)

В списки попали не только имена, но и нарицательные и начала преложений, поэтому можно найти имена которые протегирует NER модель как PERSON. Предварительно необходимо ее скачать, а также, поскольку это .jar необходима установленная java (тестировалось на openjdk 8). Здесь она лежит в корне вместе в ноутбуком. Полностью список может не уместиться в памяти, поэтому делаем обработку побатчево

In [ ]:
st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz')

In [129]:
tagged_fn = []
batch_size = 100000
flattened_fn = [j for i in Counter(full_names).keys() for j in i.split()]
for i in tqdm(range(0, len(flattened_fn), batch_size)):
    tagged_fn += st.tag(flattened_fn[i:i+batch_size])

100%|██████████| 14/14 [01:13<00:00,  5.22s/it]


In [147]:
tagged_pn = []
batch_size = 100000
flattened_pn = [j for i in Counter(professor_names).keys() for j in i.split()]
for i in tqdm(range(0, len(flattened_pn), batch_size)):
    tagged_pn += st.tag(flattened_pn[i:i+batch_size])

100%|██████████| 1/1 [00:02<00:00,  2.18s/it]


In [157]:
tagged_names = []
tagged_full_names = []
tagged_professor_names = []

counter_tagged_names = Counter()
counter_tagged_full_names = Counter()
counter_tagged_professor_names = Counter()

counter_names = Counter(names)
counter_full_names = Counter(full_names)
counter_professor_names = Counter(professor_names)

for tag in tqdm(st.tag(counter_names.keys())):
    if tag[1] == 'PERSON': counter_tagged_names[tag[0]] = counter_names[tag[0]]
for tags in tqdm(biwords(tagged_fn)):
    if tags[0][1]=='PERSON' and tags[1][1]=='PERSON':
        counter_key = ' '.join([tags[0][0], tags[1][0]])
        counter_tagged_full_names[counter_key] = counter_full_names[counter_key]
for tags in tqdm(biwords(tagged_pn)):
    if tags[0][0].lower()=='professor' and tags[1][1]=='PERSON':
        counter_key = ' '.join([tags[0][0], tags[1][0]])
        counter_tagged_professor_names[counter_key] = counter_professor_names[counter_key]

100%|██████████| 282329/282329 [00:00<00:00, 2986988.46it/s]
1311337it [00:00, 2302662.28it/s]
7177it [00:00, 1559150.56it/s]


Посмотрим что получилось:

In [159]:
counter_tagged_names.most_common(10)

[('Harry', 3881517),
 ('Draco', 1353411),
 ('Ron', 881149),
 ('Ginny', 620418),
 ('Snape', 581473),
 ('Malfoy', 472014),
 ('Dumbledore', 459590),
 ('Remus', 420104),
 ('Voldemort', 383796),
 ('Neville', 216927)]

In [162]:
counter_tagged_full_names.most_common(10)

[('Harry Potter', 124825),
 ('Draco Malfoy', 47237),
 ('Professor Snape', 42366),
 ('Miss Granger', 35790),
 ('Severus Snape', 30498),
 ('Hermione Granger', 30052),
 ('James Potter', 27171),
 ('Sirius Black', 26291),
 ('Lucius Malfoy', 23940),
 ('Albus Dumbledore', 20988)]

In [161]:
counter_tagged_professor_names.most_common(10)

[('Professor Snape', 42366),
 ('Professor Dumbledore', 22112),
 ('Professor Flitwick', 8995),
 ('Professor Trelawney', 2966),
 ('Professor Umbridge', 2745),
 ('Professor Longbottom', 2296),
 ('Professor Binns', 2023),
 ('Professor Quirrell', 1666),
 ('Professor Potter', 1454),
 ('Professor Lockhart', 1363)]

## Часть 2. Модели представления слов
1. Обучите модель представления слов (word2vec, GloVe, fastText или любую другую) на материале корпуса HPAC.

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
from gensim.models.fasttext import FastText
from gensim.models import word2vec
from gensim.utils import tokenize
from gensim import utils
from pymystem3 import Mystem
from functools import lru_cache

Вспомогательные функции для фильтрации, лемматизации и определения итератора по строкам всех файлов чтобы не загружать корпус в память. К сожалению фильтрация и лематизация делает обучение слишком долгим, поэтому пока обучаем эмбединги на необработанном корпусе

In [13]:
lemmatizer = Mystem()

@lru_cache(maxsize=128)
def lemm_word(word):
    return lemmatizer.lemmatize(word)[0]
    
def lemm(words):
    return [lemm_word(word) for word in words]

In [14]:
def filter_stopwords(tokens):
    return [word for word in tokens if word not in stopwords.words("english")]

In [15]:
def filter_punkt(tokens):
    return [token for token in tokens if not any(punct in word for punct in string.punctuation)]

In [16]:
class FullCorpusIterator():
    def __iter__(self):
        for raw_text_file in os.listdir(tokenized_corpus_path):
            with open(os.path.join(tokenized_corpus_path, raw_text_file)) as f:
                for line in f:
                    yield line.split()
#                     yield lemm(filter_stopwords_and_punkt(tokenize(line))) # unfortunatly too slow

Код для fasttext, но дальше будет обучен и использоваться Word2Vec

In [37]:
# embedder_model = FastText(size=100, min_count=1)
# epochs = 10
# embedder_model.build_vocab(sentences=FullCorpusIterator())
# embedder_model.train(
#     sentences=FullCorpusIterator(), 
#     epochs=epochs,
#     total_examples=embedder_model.corpus_count, 
#     total_words=embedder_model.corpus_total_words,
# )
# with open("embedder_model.pckl", 'wb') as file:
#     pickle.dump(embedder_model, file)
# embedder_model.wv.most_similar("strange")

2021-02-21 21:36:06,260 : INFO : resetting layer weights


Обучим Word2Vec:

In [14]:
word2vec_model = word2vec.Word2Vec(
    sentences=FullCorpusIterator(), 
    workers=8, 
    size=300, 
    min_count=10, 
    window=10, 
    sample=1e-3
)

In [44]:
# Сохранение
# word2vec_model.save("w2vmodel.model")

2021-02-21 23:38:10,799 : INFO : saving Word2Vec object under w2vmodel.model, separately None
2021-02-21 23:38:10,801 : INFO : storing np array 'vectors' to w2vmodel.model.wv.vectors.npy
2021-02-21 23:38:10,861 : INFO : not storing attribute vectors_norm
2021-02-21 23:38:10,862 : INFO : storing np array 'syn1neg' to w2vmodel.model.trainables.syn1neg.npy
2021-02-21 23:38:10,915 : INFO : not storing attribute cum_table
2021-02-21 23:38:11,026 : INFO : saved w2vmodel.model


In [7]:
# Загрузка
# word2vec_model = word2vec.Word2Vec.load("w2vmodel.model")

2021-02-25 00:55:24,017 : INFO : loading Word2Vec object from w2vmodel.model
2021-02-25 00:55:24,215 : INFO : loading wv recursively from w2vmodel.model.wv.* with mmap=None
2021-02-25 00:55:24,215 : INFO : loading vectors from w2vmodel.model.wv.vectors.npy with mmap=None
2021-02-25 00:55:24,270 : INFO : setting ignored attribute vectors_norm to None
2021-02-25 00:55:24,270 : INFO : loading vocabulary recursively from w2vmodel.model.vocabulary.* with mmap=None
2021-02-25 00:55:24,271 : INFO : loading trainables recursively from w2vmodel.model.trainables.* with mmap=None
2021-02-25 00:55:24,271 : INFO : loading syn1neg from w2vmodel.model.trainables.syn1neg.npy with mmap=None
2021-02-25 00:55:24,323 : INFO : setting ignored attribute cum_table to None
2021-02-25 00:55:24,324 : INFO : loaded w2vmodel.model


2. Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели.
Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP (https://umap-learn.readthedocs.io).

Можно посмотреть как с помощью эмбедингов работает поиск синонимов, ассоциаций и лишних слов:

In [13]:
print(word2vec_model.wv.most_similar(positive=["harry", "potter"], negative=["ron"], topn=1))
print(word2vec_model.wv.most_similar(positive=["albus", "dumbledore"], negative=["minerva"], topn=1))
print(word2vec_model.wv.most_similar("albus", topn=3))
print(word2vec_model.wv.doesnt_match("albus harry draco leviosa".split()))

2021-02-22 01:34:25,521 : INFO : precomputing L2-norms of word weight vectors


[('snape', 0.5841378569602966)]
[('headmaster', 0.5601098537445068)]
[('severus', 0.6625256538391113), ('aberforth', 0.6533681154251099), ('dumbledore', 0.6430376768112183)]
leviosa


/home/care1e55/.local/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Визуализация производится с помощью быстрой реализации t-SNE на CUDA. Преварительно ее следует установить.
Визуализируем эмбединги для подсчитанных предварительно топ 1000 слов. Визуализация с помошью bokeh

In [6]:
from tsnecuda import TSNE
import faiss
from sklearn.preprocessing import StandardScaler

2021-02-25 00:55:21,743 : INFO : Loading faiss with AVX2 support.
2021-02-25 00:55:21,744 : INFO : Loading faiss.


In [10]:
# get embedded words
embeded = []
labeles = []
# for word in tqdm(word2vec_model.wv.vocab.keys()):
for word in tqdm([i[0] for i in Counter(top_1000_filtered).most_common(1000)]):
    try:
        embeded.append(word2vec_model.wv.get_vector(word))
        labeles.append(word)
    except:
        pass
# print(embeded)

100%|██████████| 1000/1000 [00:00<00:00, 702681.19it/s]


In [11]:
# train t-sne
tsne = TSNE(n_components=2, verbose=5)
word_vectors_tsne = tsne.fit_transform(np.array(embeded))

In [12]:
# Отнормируем и выполним преобразование
ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

In [13]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=labeles)

Loading BokehJS ...

Figure(id='1003', ...)

Видно кластеры, например по частям речи

## Часть 3. Классификация текстов

1. Используйте fastText в качестве baseline-классификатора.

fasttext работает с файлом в специальном тестовом формате с __label__ перед целевой переменной, поэтому требуется предобработать датасет

In [114]:
import fasttext

In [115]:
datasets = {}
for file_path in os.listdir(split_corpus_path):
    datasets[file_path.split('_')[1]] = pd.read_csv(
        os.path.join(split_corpus_path, file_path), sep='\t', header=None
    )
    datasets[file_path.split('_')[1]].columns=['location', 'target', 'source']

In [32]:
for key, value in datasets.items():
    print(key)
    print(len(value))
    print(value.head(1))

dev
7605
          location   target                                             source
0  10873161.0.5925  PROTEGO  could do . kingsley vanished the chair that ro...
training
60980
        location      target  \
0  7642954.0.676  RIDDIKULUS   

                                              source  
0  were staring at her . she was up next to face ...  
test
7679
         location   target                                             source
0  12060772.0.457  STUPEFY  now , his eyes wide and full of apologies he h...


In [52]:
def prepare_for_fasttext(datasets):
    with open('data.train.txt', 'w+') as outfile:
        for i in range(len(datasets['training'])):
            outfile.write('__label__' + datasets['training']['target'][i] + ' '+ datasets['training']['source'][i] + '\n')

    with open('test.txt', 'w+') as outfile:
        for i in range(len(datasets['test'])):
            outfile.write('__label__' + datasets['test']['target'][i] + ' '+ datasets['test']['source'][i] + '\n')

In [ ]:
prepare_for_fasttext(datasets)

Обучим классификатор на сыром датасете (с пунктцацией и стоп-словами)

In [39]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

In [40]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.23271259278551895
R@1: 0.23271259278551895
Number of examples: 7679


Если предварительно пофильтровать стоп-слова и пункутацию, видно что качество классификации немного улучшается:

In [104]:
datasets['training']['source'] = datasets['training']['source'].str.split().apply(filter_stopwords_and_punkt)
datasets['training']['source'] = datasets['training']['source'].str.join(" ")

In [106]:
# Сохраним
# with open('train_filtered.pkl', 'wb') as f:
#     pickle.dump(datasets['training'], f)

In [107]:
# Перед вызовом функции требуется удалить предыдущие датасеты вручную
prepare_for_fasttext(datasets)

In [108]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

In [109]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.27477536137517905
R@1: 0.27477536137517905
Number of examples: 7679


Лемматицзация не дает прироста качества:

In [110]:
datasets['training']['source'] = datasets['training']['source'].str.split().apply(lemm)
datasets['training']['source'] = datasets['training']['source'].str.join(" ")

In [112]:
# Перед вызовом функции требуется удалить предыдущие датасеты вручную
prepare_for_fasttext(datasets)

In [113]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

In [114]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.27295220731866127
R@1: 0.27295220731866127
Number of examples: 7679


2. Используйте сверточные сети в качестве более продвинутого классификатора. Поэкспериментируйте с количеством и размерностью фильтров, используйте разные размеры окон, попробуйте использовать $k$-max pooling.

In [266]:
import torchtext
import torch
from torchtext import data
from torchtext import datasets
import spacy
from sklearn.preprocessing import LabelEncoder
import math
from torch.optim.lr_scheduler import StepLR

In [267]:
en_nlp = spacy.load('en')

Подготовим датасеты предварительно пофильтровав стоп-слова и пунктуацию

In [116]:
for key, value in datasets.items(): 
    value['source'] = value['source'].str.split().apply(filter_stopwords_and_punkt)
    value['source'] = value['source'].str.join(" ")

In [119]:
for key, value in datasets.items():
    value.to_csv(key+'_fitered', sep="\t", header=None, index=None)

In [22]:
for file_path in os.listdir(split_corpus_path):
    print(os.path.join(split_corpus_path, file_path))

corpus/hpac_corpus/hpac_dev_128.tsv
corpus/hpac_corpus/hpac_training_128.tsv
corpus/hpac_corpus/hpac_test_128.tsv


Для обучения используется немного модифицированный код с семинара

In [123]:
SOURCE = data.Field(tokenize='spacy')
TARGET = data.LabelField()

fields = [
    (None, None),
    ('target', TARGET),
    ('source', SOURCE)
]

train, valid, test = data.TabularDataset.splits(
    path = "",
    train = 'training_fitered',
    validation = 'dev_fitered',
    test = 'test_fitered',
    format = 'tsv',
    fields = fields,
    skip_header = False
)

Функция для подсчета multilabel accuracy:

In [268]:
import torch.nn.functional as F
def categorical_accuracy(preds, y):
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

Train loop почти без изменений взят из семинара - поправлены размерности и названия полей:

In [269]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0 
    model.train()
    for batch in tqdm(iterator, total=math.ceil(len(train)/BATCH_SIZE)):
        optimizer.zero_grad()
        predictions = model(batch.source.cuda())
        loss = criterion(predictions, batch.target.cuda())
        acc = categorical_accuracy(predictions.detach(), batch.target.detach().cuda())
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_acc += acc
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [270]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.source.cuda())
            loss = criterion(predictions, batch.target.cuda())
            acc = categorical_accuracy(predictions, batch.target.cuda())
            epoch_loss += loss
            epoch_acc += acc
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Используем w2v эмбединги обученные ранее для представления слов. Для этого сохраним в текстовом формате и построим словарь: 

In [285]:
word2vec_model = word2vec.Word2Vec.load("w2vmodel.model")
word2vec_model.wv.save_word2vec_format('w2v.txt')

In [272]:
from torchtext.vocab import Vectors
vectors = Vectors(name='w2v.txt', cache='./')

In [142]:
# SOURCE.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
SOURCE.build_vocab(train, max_size=25000, vectors=vectors)
TARGET.build_vocab(train)

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.source), 
    repeat=False)

/home/care1e55/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [143]:
# view for debug
# print(TARGET.vocab.stoi)
# TARGET.vocab.freqs

Сверточная сеть с семинара. Изменения в размерности выходного слоя, т.к. теперь классификация не бинарная:

In [275]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels = 1, 
                out_channels = n_filters, 
                kernel_size = (fs, embedding_dim)
            ) 
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [sent len, batch size]
        text = text.permute(1, 0)
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [151]:
INPUT_DIM = len(SOURCE.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(TARGET.vocab)
DROPOUT = 0.2

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

Для улучшения сходимости используем stepLR scheduler.
Используем CrossEntropyLoss, которая уже содержит в себе softmax:

In [152]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
criterion = nn.CrossEntropyLoss()
model = model.cuda()

In [153]:
N_EPOCHS = 15

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    scheduler.step()
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 2/477 [00:00<00:28, 16.43it/s]

Epoch: 01, Train Loss: 3.443, Train Acc: 16.64%, Val. Loss: 3.136, Val. Acc: 22.82%


  0%|          | 2/477 [00:00<00:30, 15.83it/s]

Epoch: 02, Train Loss: 3.123, Train Acc: 23.39%, Val. Loss: 3.012, Val. Acc: 25.32%


  0%|          | 2/477 [00:00<00:29, 16.02it/s]

...

Epoch: 13, Train Loss: 2.004, Train Acc: 46.19%, Val. Loss: 2.871, Val. Acc: 28.92%


  0%|          | 2/477 [00:00<00:29, 15.95it/s]

Epoch: 14, Train Loss: 1.984, Train Acc: 46.70%, Val. Loss: 2.875, Val. Acc: 28.78%


100%|██████████| 477/477 [00:20<00:00, 23.63it/s]


Epoch: 15, Train Loss: 1.972, Train Acc: 46.92%, Val. Loss: 2.879, Val. Acc: 28.95%


In [154]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

Test Loss: 2.856, Test Acc: 30.33%


Видно, что получили качество лучше, чем baseline

3. [2 балла] Попробуйте расширить обучающее множество за счет аугментации данных. Если вам понадобится словарь синонимов, можно использовать WordNet (ниже вы найдете примеры).

In [239]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import random

[nltk_data] Downloading package wordnet to /home/care1e55/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Подготовим дообработанные датасеты

In [244]:
datasets = {}
for file_path in ['training_fitered', 'dev_fitered', 'test_fitered']:
    datasets[file_path] = pd.read_csv(
        file_path, sep='\t', header=None
    )
    datasets[file_path].columns=['location', 'target', 'source']

Функция для замены из wordnet и расширения датасета. Датасет будет содержить и оригинал, в остальных случаях если замена возможно - выбрать случайно из списка замен:

In [258]:
def synonims(row):
    result = []
    for word in row.split():
        try:
            result.append(random.choice(random.choice(wn.synsets(word)).lemmas()).name())
        except:
            result.append(word)
    return " ".join(result)

In [259]:
def extend_dataset(dataset, factor=3):
    for i in tqdm(range(factor)):
        if i == 0:
            result = dataset.copy(deep=True)
            continue
        current_dataset = dataset.copy(deep=True)
        current_dataset['source'] = current_dataset['source'].apply(synonims)
        result = result.append(current_dataset, ignore_index=True)
    return result

Убедимся что функция работает:

In [261]:
debug_test_dataset = extend_dataset(datasets['dev_fitered'].dropna())

100%|██████████| 3/3 [00:08<00:00,  2.74s/it]


In [262]:
debug_test_dataset.shape
print(debug_test_dataset.loc[0])
print(debug_test_dataset.loc[7594])
print(debug_test_dataset.loc[2*7594])

location                                      10873161.0.5925
target                                                PROTEGO
source      could kingsley vanished chair ronald sat erect...
Name: 0, dtype: object
location                                      10873161.0.5925
target                                                PROTEGO
source      could kingsley vanished chair ronald Sat rear ...
Name: 7594, dtype: object
location                                      10873161.0.5925
target                                                PROTEGO
source      could kingsley fly chair ronald sit put_up pro...
Name: 15188, dtype: object


Применим с train выборке:

In [264]:
new_train = extend_dataset(datasets['training_fitered'].dropna())

100%|██████████| 3/3 [01:10<00:00, 23.54s/it]


In [265]:
new_train.to_csv('new_training_fitered', sep="\t", header=None, index=None)

Проведем эксперимент обучив ту же модель с новым датасетом. Модель переобучается после 3ей эпохи:

In [273]:
SOURCE = data.Field(tokenize='spacy')
TARGET = data.LabelField()

fields = [
    (None, None),
    ('target', TARGET),
    ('source', SOURCE)
]

train, valid, test = data.TabularDataset.splits(
    path = "",
    train = 'new_training_fitered',
    validation = 'dev_fitered',
    test = 'test_fitered',
    format = 'tsv',
    fields = fields,
    skip_header = False
)

/home/care1e55/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/care1e55/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/care1e55/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see

In [280]:
# SOURCE.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
SOURCE.build_vocab(train, max_size=25000, vectors=vectors)
TARGET.build_vocab(train)

BATCH_SIZE = 256

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.source), 
    repeat=False)

In [281]:
INPUT_DIM = len(SOURCE.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(TARGET.vocab)
DROPOUT = 0.2

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [282]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
criterion = nn.CrossEntropyLoss()
model = model.cuda()

In [283]:
N_EPOCHS = 3

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    scheduler.step()
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/714 [00:00<?, ?it/s]

Epoch: 01, Train Loss: 3.164, Train Acc: 21.90%, Val. Loss: 2.954, Val. Acc: 27.43%


  0%|          | 1/714 [00:00<02:17,  5.17it/s]

Epoch: 02, Train Loss: 2.840, Train Acc: 28.72%, Val. Loss: 2.858, Val. Acc: 29.76%


100%|██████████| 714/714 [00:47<00:00, 15.16it/s]


Epoch: 03, Train Loss: 2.628, Train Acc: 32.67%, Val. Loss: 2.830, Val. Acc: 30.29%


In [284]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

Test Loss: 2.790, Test Acc: 30.59%


Можно сделать вывод, что качество примерно остается тем же, но улучшается скорость сходимости

## Часть 4. Итоги

Проведен эксплоративный анализ и подсчитаны частоты слов и имен. Для корректного поиска именованых сущностей использвана модель Stanford NER.
Произведена базовая обработка датасета - удалены стоп-слова и пунктуация, сделана лемматизация.
Обучена модель представления слов word2vec с помощью пакета gensim, продемонстрировано как работает поиск синонимов, ассоциаций, лишних слов в обученной модели, а также получена визуализация пространства эмбедингов наиболее часто встречающихся слов с помощью CUDA-реализации алгоритма t-SNE.
Обучена baseline модель fasstext для классификации, продемонстрировано улучшения качества после препроцессинга датасета. Обучена сверточная нейронная сеть для решенеия задачи мультиклассовой классификации, которая показала лучше качество, чем baseline. Датасет расширен аугментированными данными. Возможно дальнейшее улучшение модели, например использование LSTM или LSTM+CNN, Transformer.
При решении задачи немного помогло знание предметной области при опредлении валидных именованых сущностей, но, кажется знание предметной области не обязательно для решения задачи.